In [1]:
#importing libraries
import spacy
from spacy.vocab import Vocab
import numpy
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, Bidirectional
import pickle

In [2]:
#reading processed data
data = open('basededados.csv').read()[:1000000]
data

'0\n"  Aprenda a preparar um delicioso pão caseiro recheado Neste domingo (18), o Clube Rural traz a receita de um pão caseiro. A cozinheira Cynthia Ribeiro ensina a fazer esse prato delicioso e fácil de ser preparado.       Pão Caseiro — Foto: Reprodução/TV Clube     Comédia Pão com Ovo faz live beneficente no dia 20 de junho. — Foto: Divulgação. No dia 20 de junho a família Pão com Ovo vai invadir as telas, a partir das 14h30, com uma live divertida e beneficente. A transmissão ao vivo será feita pelo Gshow e pela TV Mirante, afiliada da Rede Globo, que inova mais uma vez e traz um conteúdo diferente para alegrar o telespectador maranhense. O grupo de comédia vai levar muito humor e ainda arrecadar doações para os maranhenses que mais precisam de auxílio neste momento de pandemia. A diversão vai ser garantida, já que Yardi, filha de Dijé, planeja um noivado online no bairro do Fumacê, em São Luís. E Clarisse aproveita a oportunidade para ver esse fuxico de perto e se voluntaria para 

In [3]:
#function for preparing text data into sequences for training 
def data_sequencing(data):   
    # integer encode sequences of words
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([data])
    with open('tokenizer.pkl', 'wb') as f: # Save the tokeniser by pickling it
        pickle.dump(tokenizer, f)

    encoded = tokenizer.texts_to_sequences([data])[0]
    # retrieve vocabulary size
    vocab_size = len(tokenizer.word_index) + 1
    print('Vocabulary Size: %d' % vocab_size)
    
    # create line-based sequences
    sequences = list()
    rev_sequences = list()
    for line in data.split('.'):
        encoded = tokenizer.texts_to_sequences([line])[0]
        rev_encoded = encoded[::-1]
        for i in range(1, len(encoded)):
            sequence = encoded[:i+1]
            rev_sequence = rev_encoded[:i+1]
            sequences.append(sequence)
            rev_sequences.append(rev_sequence)
    print('Total Sequences: %d' % len(sequences))
    
    
    #find max sequence length 
    max_length = max([len(seq) for seq in sequences])
    with open('max_length.pkl', 'wb') as f: # Save max_length by pickling it
        pickle.dump(max_length, f)
    print('Max Sequence Length: %d' % max_length)

    # pad sequences and create the forward sequence
    sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
    # split into input and output elements
    sequences = array(sequences)
    X, y = sequences[:,:-1],sequences[:,-1]
    
    #pad sequences and create the reverse sequencing
    rev_sequences = pad_sequences(rev_sequences, maxlen=max_length, padding='pre')
    # split into input and output elements
    rev_sequences = array(rev_sequences)
    rev_X, rev_y = rev_sequences[:,:-1],rev_sequences[:,-1]

    return X,y,rev_X,rev_y,max_length,vocab_size

In [4]:
#returning forward and reverse sequences along with max sequence 
#length from the data 

X,y,rev_X,rev_y,max_length,vocab_size = data_sequencing(data)

Vocabulary Size: 17610
Total Sequences: 160880
Max Sequence Length: 2010


In [5]:
# define forward sequence model
model = Sequential()
model.add(Embedding(vocab_size,100, input_length=max_length-1))
#model.add(LSTM(100))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2009, 100)         1761000   
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 17610)             3539610   
Total params: 5,461,410
Trainable params: 5,461,410
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# define reverse model
rev_model = Sequential()
rev_model.add(Embedding(vocab_size, 100, input_length=max_length-1))
#rev_model.add(LSTM(100))
rev_model.add(Bidirectional(LSTM(100)))
rev_model.add(Dense(vocab_size, activation='softmax'))
print(rev_model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2009, 100)         1761000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_1 (Dense)              (None, 17610)             3539610   
Total params: 5,461,410
Trainable params: 5,461,410
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
# compile forward sequence network
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y,batch_size=100, epochs=200, verbose=2)
# save the model to file
model.save('model.h5')

Epoch 1/200
1609/1609 - 4609s - loss: 7.3475 - accuracy: 0.0620
Epoch 2/200
1609/1609 - 4503s - loss: 6.5479 - accuracy: 0.0924
Epoch 3/200
1609/1609 - 4509s - loss: 6.0043 - accuracy: 0.1204
Epoch 4/200
1609/1609 - 4502s - loss: 5.5593 - accuracy: 0.1463
Epoch 5/200
1609/1609 - 4500s - loss: 5.1707 - accuracy: 0.1730
Epoch 6/200
1609/1609 - 4527s - loss: 4.8197 - accuracy: 0.1978
Epoch 7/200
1609/1609 - 4533s - loss: 4.4947 - accuracy: 0.2252
Epoch 8/200
1609/1609 - 4613s - loss: 4.1906 - accuracy: 0.2531
Epoch 9/200
1609/1609 - 4578s - loss: 3.9056 - accuracy: 0.2848
Epoch 10/200
1609/1609 - 4572s - loss: 3.6456 - accuracy: 0.3174
Epoch 11/200
1609/1609 - 4575s - loss: 3.4065 - accuracy: 0.3518
Epoch 12/200
1609/1609 - 4586s - loss: 3.1915 - accuracy: 0.3853
Epoch 13/200
1609/1609 - 4602s - loss: 2.9974 - accuracy: 0.4171
Epoch 14/200
1609/1609 - 4562s - loss: 2.8223 - accuracy: 0.4452
Epoch 15/200
1609/1609 - 4586s - loss: 2.6626 - accuracy: 0.4731
Epoch 16/200
1609/1609 - 4591s - l

Epoch 127/200
1609/1609 - 4692s - loss: 0.3103 - accuracy: 0.9283
Epoch 128/200
1609/1609 - 4695s - loss: 0.3139 - accuracy: 0.9268
Epoch 129/200
1609/1609 - 4708s - loss: 0.3132 - accuracy: 0.9271
Epoch 130/200
1609/1609 - 4705s - loss: 0.3068 - accuracy: 0.9288
Epoch 131/200
1609/1609 - 4703s - loss: 0.3037 - accuracy: 0.9290
Epoch 132/200
1609/1609 - 4622s - loss: 0.3060 - accuracy: 0.9288
Epoch 133/200
1609/1609 - 4624s - loss: 0.3054 - accuracy: 0.9289
Epoch 134/200
1609/1609 - 4624s - loss: 0.3047 - accuracy: 0.9287
Epoch 135/200
1609/1609 - 4652s - loss: 0.3026 - accuracy: 0.9291
Epoch 136/200
1609/1609 - 4634s - loss: 0.3033 - accuracy: 0.9286
Epoch 137/200
1609/1609 - 4646s - loss: 0.2989 - accuracy: 0.9303
Epoch 138/200
1609/1609 - 5293s - loss: 0.2993 - accuracy: 0.9299
Epoch 139/200
1609/1609 - 5114s - loss: 0.3003 - accuracy: 0.9293
Epoch 140/200
1609/1609 - 4868s - loss: 0.2971 - accuracy: 0.9301
Epoch 141/200
1609/1609 - 4656s - loss: 0.2944 - accuracy: 0.9306
Epoch 142/

In [8]:
# compile reverse sequence network
rev_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
rev_model.fit(rev_X, rev_y,batch_size=100, epochs=200, verbose=2)
# save the model to file
rev_model.save('rev_model.h5')

Epoch 1/200
1609/1609 - 4663s - loss: 7.1690 - accuracy: 0.0673
Epoch 2/200
1609/1609 - 4684s - loss: 6.3909 - accuracy: 0.1181
Epoch 3/200
1609/1609 - 4662s - loss: 5.8161 - accuracy: 0.1617
Epoch 4/200
1609/1609 - 4580s - loss: 5.3582 - accuracy: 0.1966
Epoch 5/200
1609/1609 - 4577s - loss: 4.9773 - accuracy: 0.2246
Epoch 6/200
1609/1609 - 4573s - loss: 4.6424 - accuracy: 0.2501
Epoch 7/200
1609/1609 - 4580s - loss: 4.3371 - accuracy: 0.2760
Epoch 8/200
1609/1609 - 4576s - loss: 4.0517 - accuracy: 0.3022
Epoch 9/200
1609/1609 - 4583s - loss: 3.7858 - accuracy: 0.3307
Epoch 10/200
1609/1609 - 4581s - loss: 3.5370 - accuracy: 0.3594
Epoch 11/200
1609/1609 - 4607s - loss: 3.3024 - accuracy: 0.3904
Epoch 12/200
1609/1609 - 4637s - loss: 3.0876 - accuracy: 0.4224
Epoch 13/200
1609/1609 - 4691s - loss: 2.8925 - accuracy: 0.4537
Epoch 14/200
1609/1609 - 4729s - loss: 2.7135 - accuracy: 0.4832
Epoch 15/200
1609/1609 - 4718s - loss: 2.5491 - accuracy: 0.5123
Epoch 16/200
1609/1609 - 4720s - l

Epoch 127/200
1609/1609 - 4870s - loss: 0.1601 - accuracy: 0.9550
Epoch 128/200
1609/1609 - 4840s - loss: 0.1556 - accuracy: 0.9560
Epoch 129/200
1609/1609 - 4859s - loss: 0.1561 - accuracy: 0.9559
Epoch 130/200
1609/1609 - 4825s - loss: 0.1623 - accuracy: 0.9543
Epoch 131/200
1609/1609 - 4738s - loss: 0.1542 - accuracy: 0.9560
Epoch 132/200
1609/1609 - 4732s - loss: 0.1520 - accuracy: 0.9565
Epoch 133/200
1609/1609 - 4745s - loss: 0.1531 - accuracy: 0.9564
Epoch 134/200
1609/1609 - 4734s - loss: 0.1512 - accuracy: 0.9569
Epoch 135/200
1609/1609 - 4733s - loss: 0.1528 - accuracy: 0.9559
Epoch 136/200
1609/1609 - 4744s - loss: 0.1530 - accuracy: 0.9557
Epoch 137/200
1609/1609 - 4743s - loss: 0.1467 - accuracy: 0.9578
Epoch 138/200
1609/1609 - 4736s - loss: 0.1449 - accuracy: 0.9578
Epoch 139/200
1609/1609 - 4769s - loss: 0.1496 - accuracy: 0.9563
Epoch 140/200
1609/1609 - 4913s - loss: 0.1493 - accuracy: 0.9564
Epoch 141/200
1609/1609 - 4825s - loss: 0.1459 - accuracy: 0.9577
Epoch 142/

In [7]:
# generate a sequence using a language model
def generate_seq(model, tokenizer, max_length, seed_text):
    if seed_text == "":
        return ""
    else:
        in_text = seed_text
        n_words = 1
        n_preds = 5 #number of words to predict for the seed text
        pred_words = ""
        # generate a fixed number of words
        for _ in range(n_words):
            # encode the text as integer
            encoded = tokenizer.texts_to_sequences([in_text])[0]
            # pre-pad sequences to a fixed length
            encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
            # predict probabilities for each word
            proba = model.predict(encoded, verbose=0).flatten()
            #take the n_preds highest probability classes 
            yhat = numpy.argsort(-proba)[:n_preds] 
            # map predicted words index to word
            out_word = ''

            for _ in range(n_preds):
                for word, index in tokenizer.word_index.items():
                    if index == yhat[_] and word not in stopwords:
                        out_word = word
                        pred_words += ' ' + out_word
                        #print(out_word)
                        break


        return pred_words

In [8]:
# load the model
model = load_model('model.h5')
rev_model = load_model('rev_model.h5')

#load tokeniser and max_length
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
    
with open('max_length.pkl', 'rb') as f:
    max_length = pickle.load(f)
    
#loading stopwords to improve relevant word predictions    
stopwords= open('stopwords_pt.txt').read().split()

#load spacy GloVe Model
nlp = spacy.load('pt_core_news_md')

In [9]:
#Find and set embeddings for OOV words
def set_embedding_for_oov(doc):
    #checking for oov words and adding embedding
    for token in doc:
        if token.is_oov == True:
            before_text = doc[:token.i].text
            after_text = str(array(doc)[:token.i:-1]).replace('[','').replace(']','')

            pred_before = generate_seq(model, tokenizer, max_length-1, before_text).split()
            pred_after = generate_seq(rev_model, tokenizer, max_length-1, after_text).split()
            
            embedding = numpy.zeros((300,))

            i=len(before_text)
            print('Words predicted from forward sequence model:')
            for word in pred_before:
                print(word)
                embedding += i*nlp.vocab.get_vector(word)
                i= i*.5
            i=len(after_text)
            print('Words predicted from reverse sequence model:')
            for word in pred_after:
                print(word)
                embedding += i*nlp.vocab.get_vector(word)
                i= i*.5
            nlp.vocab.set_vector(token.text, embedding)
            print(token.text,nlp.vocab.get_vector(token.text))        

In [10]:
#function to find most similar words
def most_similar(word):
    by_similarity = sorted(word.vocab, key=lambda w: word.similarity(w), reverse=True)
    return [w.orth_ for w in by_similarity[:10]]

# Teste 1

In [11]:
doc1 = nlp('Que cara esparroso! Eu nem conheço ele!')
set_embedding_for_oov(doc1)

Words predicted from forward sequence model:
dentro
deu
Words predicted from reverse sequence model:
onde
dizia
nascimento
dinheiro
esparroso [ 2.27275238e+01 -5.15735283e+01 -4.64351387e+01  9.98990784e+01
 -5.15393600e+01 -6.26485291e+01 -4.50116043e+01  9.24019089e+01
 -4.36822968e+01  6.58312225e+01 -4.20415268e+01 -1.04246979e+02
  8.84931412e+01 -4.89337311e+01  1.89014263e+01  2.24119205e+01
  1.56379547e+01  3.83292351e+01 -1.02431648e+02  1.06412331e+02
 -1.80931244e+01  3.42931900e+01 -3.94329491e+01  1.28087891e+02
  2.33941956e+01  6.81264038e+01 -5.66610451e+01  1.56608948e+02
  3.63015060e+01 -2.92840805e+01 -1.36084099e+01  1.30243744e+02
  1.46374817e+01 -9.32423687e+00 -2.72992077e+01  1.70413895e+01
  2.66497211e+01  2.07034492e+01  8.27309113e+01  3.52598267e+01
  1.77859306e+01  1.10793083e+02  3.89833946e+01  7.27636795e+01
  1.27274933e+01 -9.43133698e+01  2.42996063e+01 -7.34295654e+01
  9.12689972e+01  2.21709404e+01  6.28428459e+01  1.10436745e+01
  2.16756554e

In [12]:
doc2 = nlp('Ele é muito esparroso!')
set_embedding_for_oov(doc2)

In [13]:
doc3 = nlp('Se ele continuar sendo esparroso desse jeito, ele vai se dá mal!')
set_embedding_for_oov(doc3)

In [14]:
doc4 = nlp('E pense num qualira esparroso!')
set_embedding_for_oov(doc4)

Words predicted from forward sequence model:
lugar
cenário
26
post
tempo
Words predicted from reverse sequence model:
ano
novamente
oficial
uti
feira
qualira [-2.06378078e+01 -1.36855793e+01 -6.43589401e+01  1.52412395e+01
  1.11192657e+02 -1.01076256e+02 -8.94587402e+01 -6.89768524e+01
 -2.68960171e+01  1.07148193e+02  1.24925842e+01  5.97369118e+01
  1.26658525e+01 -1.72663422e+01  1.06519260e+01 -7.72578964e+01
 -6.85203094e+01  6.08105736e+01  1.05136080e+01 -1.33378327e-01
  4.09758043e+00 -4.01997147e+01  3.07599525e+01 -1.65893917e+01
  6.44056091e+01 -7.35429811e+00 -9.80649338e+01  1.60482086e+02
 -1.37295313e+01 -1.02025307e+02  3.51260071e+01  1.06442245e+02
  1.40533615e+02 -6.86794586e+01  9.15516663e+01 -8.78833580e+00
 -1.11255226e+01 -9.57491207e+00 -8.23221302e+00 -1.23166084e+02
  7.04622192e+01  8.52110443e+01  2.06634159e+01  8.87803936e+00
  9.64204941e+01 -9.50158119e-01  2.55518322e+01 -3.05894260e+01
  1.35339565e+01  4.65330658e+01  7.83097000e+01  3.40339737e+

In [15]:
nlp('esparroso').similarity(nlp('exagerado'))

0.18405502761727702

In [16]:
most_similar(nlp('esparroso'))

/home/paulamyrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  This is separate from the ipykernel package so we can avoid doing imports until


['esparroso',
 'onde',
 'dentro',
 'novamente',
 'lugar',
 '^__^',
 ':-/',
 'num',
 'dizia',
 'sra']

In [17]:
nlp.vocab.get_vector('esparroso')

array([ 2.27275238e+01, -5.15735283e+01, -4.64351387e+01,  9.98990784e+01,
       -5.15393600e+01, -6.26485291e+01, -4.50116043e+01,  9.24019089e+01,
       -4.36822968e+01,  6.58312225e+01, -4.20415268e+01, -1.04246979e+02,
        8.84931412e+01, -4.89337311e+01,  1.89014263e+01,  2.24119205e+01,
        1.56379547e+01,  3.83292351e+01, -1.02431648e+02,  1.06412331e+02,
       -1.80931244e+01,  3.42931900e+01, -3.94329491e+01,  1.28087891e+02,
        2.33941956e+01,  6.81264038e+01, -5.66610451e+01,  1.56608948e+02,
        3.63015060e+01, -2.92840805e+01, -1.36084099e+01,  1.30243744e+02,
        1.46374817e+01, -9.32423687e+00, -2.72992077e+01,  1.70413895e+01,
        2.66497211e+01,  2.07034492e+01,  8.27309113e+01,  3.52598267e+01,
        1.77859306e+01,  1.10793083e+02,  3.89833946e+01,  7.27636795e+01,
        1.27274933e+01, -9.43133698e+01,  2.42996063e+01, -7.34295654e+01,
        9.12689972e+01,  2.21709404e+01,  6.28428459e+01,  1.10436745e+01,
        2.16756554e+01,  

# Teste 2

In [18]:
doc5 = nlp('Estou aziado para sair de casa!')
set_embedding_for_oov(doc5)

Words predicted from forward sequence model:
fazendo
quente
Words predicted from reverse sequence model:
usado
aqui
vai
esposo
motivo
aziado [ 7.38866653e+01 -7.27138290e+01 -7.00713348e+01  1.12017128e+02
 -1.14255936e+02 -3.70572853e+01 -3.57594986e+01 -8.86938477e+01
  6.82036352e+00  8.00099254e-01  1.08787613e+02 -5.96577301e+01
  4.00572853e+01 -1.33261932e+02 -3.87268219e+01  1.35925806e+00
  1.02436275e+01 -4.78661690e+01 -6.37628441e+01  2.61610584e+01
 -1.95571651e+01  8.21447754e+01  6.20743217e+01  1.01104668e+02
  1.43410461e+02  2.15189056e+01 -6.73948975e+01  3.08388691e+01
 -6.14117470e+01  1.96709037e-01  1.43984337e+01 -4.87199364e+01
 -1.07926132e+02  2.25072479e+00  2.64457607e+01  5.23922806e+01
 -1.03073029e+02  1.01226410e+02 -6.39127693e+01  5.75940437e+01
 -1.18378937e+02  9.05955029e+00  3.94929008e+01 -1.46136436e+01
 -4.10814781e+01 -1.32383614e+01 -5.08471146e+01 -6.55834351e+01
 -5.97334709e+01 -3.81793213e+01  2.64000568e+01 -2.85851231e+01
 -6.99964294e+

In [19]:
doc6 = nlp('Este lugar está muito aziado, vou já embora!')
set_embedding_for_oov(doc6)

In [20]:
doc7 = nlp('Fulano está aziado para festa hoje.')
set_embedding_for_oov(doc7)

In [21]:
doc8 = nlp('Ontem fiquei aziado de tanto ouvir minha mãe reclamar das minhas notas.')
set_embedding_for_oov(doc8)

In [22]:
nlp('aziado').similarity(nlp('desanimado'))

0.37838483803493966

In [23]:
nlp('aziado').similarity(nlp('depressivo'))

0.1278518107617255

In [24]:
most_similar(nlp('aziado'))

/home/paulamyrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  This is separate from the ipykernel package so we can avoid doing imports until


['aziado',
 'usado',
 'exagerado',
 'vai',
 'fazendo',
 'para',
 'quente',
 'desanimado',
 'vou',
 'aqui']

In [25]:
nlp.vocab.get_vector('aziado')

array([ 7.38866653e+01, -7.27138290e+01, -7.00713348e+01,  1.12017128e+02,
       -1.14255936e+02, -3.70572853e+01, -3.57594986e+01, -8.86938477e+01,
        6.82036352e+00,  8.00099254e-01,  1.08787613e+02, -5.96577301e+01,
        4.00572853e+01, -1.33261932e+02, -3.87268219e+01,  1.35925806e+00,
        1.02436275e+01, -4.78661690e+01, -6.37628441e+01,  2.61610584e+01,
       -1.95571651e+01,  8.21447754e+01,  6.20743217e+01,  1.01104668e+02,
        1.43410461e+02,  2.15189056e+01, -6.73948975e+01,  3.08388691e+01,
       -6.14117470e+01,  1.96709037e-01,  1.43984337e+01, -4.87199364e+01,
       -1.07926132e+02,  2.25072479e+00,  2.64457607e+01,  5.23922806e+01,
       -1.03073029e+02,  1.01226410e+02, -6.39127693e+01,  5.75940437e+01,
       -1.18378937e+02,  9.05955029e+00,  3.94929008e+01, -1.46136436e+01,
       -4.10814781e+01, -1.32383614e+01, -5.08471146e+01, -6.55834351e+01,
       -5.97334709e+01, -3.81793213e+01,  2.64000568e+01, -2.85851231e+01,
       -6.99964294e+01,  

# Teste 3

In [26]:
doc9 = nlp('Deixa de viadagem, seu qualira!')
set_embedding_for_oov(doc9)

In [27]:
doc10 = nlp('Olha que qualira bonito!')
set_embedding_for_oov(doc10)

In [28]:
doc11 = nlp('Mas tú é viado, fresco ou qualira?')
set_embedding_for_oov(doc11)

In [29]:
doc12 = nlp('Aquilo alí é qualira!')
set_embedding_for_oov(doc12)

In [30]:
nlp('qualira').similarity(nlp('homossexual'))

0.03455031594459282

In [31]:
nlp('qualira').similarity(nlp('fresco'))

0.0967172009422662

In [32]:
nlp('qualira').similarity(nlp('gay'))

-0.04884627011257295

In [33]:
most_similar(nlp('qualira'))

/home/paulamyrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  This is separate from the ipykernel package so we can avoid doing imports until


['qualira',
 'ano',
 'lugar',
 'cenário',
 'tempo',
 'novamente',
 'post',
 'festa',
 'motivo',
 'feira']

In [34]:
nlp.vocab.get_vector('qualira')

array([-2.06378078e+01, -1.36855793e+01, -6.43589401e+01,  1.52412395e+01,
        1.11192657e+02, -1.01076256e+02, -8.94587402e+01, -6.89768524e+01,
       -2.68960171e+01,  1.07148193e+02,  1.24925842e+01,  5.97369118e+01,
        1.26658525e+01, -1.72663422e+01,  1.06519260e+01, -7.72578964e+01,
       -6.85203094e+01,  6.08105736e+01,  1.05136080e+01, -1.33378327e-01,
        4.09758043e+00, -4.01997147e+01,  3.07599525e+01, -1.65893917e+01,
        6.44056091e+01, -7.35429811e+00, -9.80649338e+01,  1.60482086e+02,
       -1.37295313e+01, -1.02025307e+02,  3.51260071e+01,  1.06442245e+02,
        1.40533615e+02, -6.86794586e+01,  9.15516663e+01, -8.78833580e+00,
       -1.11255226e+01, -9.57491207e+00, -8.23221302e+00, -1.23166084e+02,
        7.04622192e+01,  8.52110443e+01,  2.06634159e+01,  8.87803936e+00,
        9.64204941e+01, -9.50158119e-01,  2.55518322e+01, -3.05894260e+01,
        1.35339565e+01,  4.65330658e+01,  7.83097000e+01,  3.40339737e+01,
        4.25702324e+01,  